# Azure AI-agenter med støtte for Model Context Protocol (MCP) - Python

Denne notatboken viser hvordan du kan bruke Azure AI-agenter med Model Context Protocol (MCP)-verktøy i Python. Den demonstrerer hvordan du kan opprette en intelligent agent som kan dra nytte av eksterne MCP-servere (som Microsoft Learn) for forbedrede funksjoner ved bruk av nøkkelfri autentisering.


## Installer nødvendige Python-pakker

Først må vi installere de nødvendige Python-pakkene:
- **azure-ai-projects**: Kjerne-SDK for Azure AI Projects
- **azure-ai-agents**: Azure AI Agents SDK for å opprette og administrere agenter
- **azure-identity**: Gir nøkkelfri autentisering ved bruk av DefaultAzureCredential
- **mcp**: Implementasjon av Model Context Protocol for Python


## Fordeler med nøkkelfri autentisering

Denne notatboken demonstrerer **nøkkelfri autentisering**, som gir flere fordeler:
- ✅ **Ingen API-nøkler å administrere** - Bruker Azure-identitetsbasert autentisering
- ✅ **Forbedret sikkerhet** - Ingen hemmeligheter lagres i kode eller konfigurasjonsfiler
- ✅ **Automatisk rotasjon av legitimasjon** - Azure håndterer livssyklushåndtering av legitimasjon
- ✅ **Rollebasert tilgangskontroll** - Bruker Azure RBAC for detaljert tilgangsstyring
- ✅ **Støtte for flere miljøer** - Fungerer sømløst på tvers av utvikling og produksjon

`DefaultAzureCredential` velger automatisk den beste tilgjengelige legitimasjonskilden:
1. **Administrert identitet** (når den kjører i Azure)
2. **Azure CLI** legitimasjon (under lokal utvikling)
3. **Visual Studio** legitimasjon
4. **Miljøvariabler** (hvis konfigurert)
5. **Interaktiv nettleser** autentisering (som reserve)


## Oppsett for nøkkelfri autentisering

**Forutsetninger for nøkkelfri autentisering:**

### For lokal utvikling:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### For Azure-miljøer:
- Aktiver **System-assigned Managed Identity** på din Azure-ressurs
- Tilordne passende **RBAC-roller** til den administrerte identiteten:
  - `Cognitive Services OpenAI User` for tilgang til Azure OpenAI
  - `AI Developer` for tilgang til Azure AI-prosjekter

### Miljøvariabler (Valgfritt):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Ingen API-nøkler eller tilkoblingsstrenger nødvendig!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importer nødvendige biblioteker

Importer de nødvendige Python-modulene:
- **os, time**: Standard Python-biblioteker for miljøvariabler og forsinkelser
- **AIProjectClient**: Hovedklienten for Azure AI-prosjekter
- **DefaultAzureCredential**: Nøkkelfri autentisering for Azure-tjenester
- **MCP-relaterte klasser**: For å opprette og administrere MCP-verktøy og håndtere godkjenninger


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfigurer MCP-serverinnstillinger

Sett opp MCP-serverkonfigurasjonen ved hjelp av miljøvariabler med standardverdier som reserve:
- **MCP_SERVER_URL**: URL-en til MCP-serveren (standard er Microsoft Learn API)
- **MCP_SERVER_LABEL**: En etikett for å identifisere MCP-serveren (standard er "mslearn")

Denne tilnærmingen gir mulighet for fleksibel konfigurasjon på tvers av ulike miljøer.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Opprett Azure AI Project-klient (Nøkkelfri autentisering)

Initialiser Azure AI Project-klienten ved å bruke **nøkkelfri autentisering**:
- **endpoint**: URL-en til Azure AI Foundry-prosjektets endepunkt
- **credential**: Bruker `DefaultAzureCredential()` for sikker, nøkkelfri autentisering
- **Ingen API-nøkler nødvendig**: Oppdager og bruker automatisk den beste tilgjengelige legitimasjonen

**Autentiseringsflyt:**
1. Sjekker etter Managed Identity (i Azure-miljøer)
2. Faller tilbake på Azure CLI-legitimasjon (for lokal utvikling)
3. Bruker andre tilgjengelige legitimasjonskilder ved behov

Denne tilnærmingen eliminerer behovet for å håndtere API-nøkler eller tilkoblingsstrenger i koden din.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Opprett MCP-verktøydefinisjon

Opprett et MCP-verktøy som kobler til Microsoft Learn MCP-serveren:
- **server_label**: Identifikator for MCP-serveren
- **server_url**: URL-endepunkt for MCP-serveren
- **allowed_tools**: Valgfri liste for å begrense hvilke verktøy som kan brukes (tom liste tillater alle verktøy)

Dette verktøyet vil gjøre det mulig for agenten å få tilgang til Microsoft Learn-dokumentasjon og ressurser.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Opprett Agent og Utfør Samtale (Nøkkelfri Arbeidsflyt)

Denne omfattende seksjonen viser den komplette **nøkkelfrie agentarbeidsflyten**:

1. **Opprett AI-agent**: Sett opp en agent med GPT-4.1 nano-modell og MCP-verktøy
2. **Opprett Tråd**: Etabler en samtaletråd for kommunikasjon
3. **Send Melding**: Spør agenten om forskjellene mellom Azure OpenAI og OpenAI
4. **Håndter Verktøy-Godkjenninger**: Godkjenn MCP-verktøykall automatisk når det er nødvendig
5. **Overvåk Utførelse**: Følg med på agentens fremgang og håndter eventuelle nødvendige handlinger
6. **Vis Resultater**: Vis samtalen og detaljer om verktøybruk

**Nøkkelfrie Funksjoner:**
- ✅ **Ingen hardkodede hemmeligheter** - All autentisering håndteres av Azure-identitet
- ✅ **Sikker som standard** - Bruker rollebasert tilgangskontroll
- ✅ **Forenklet distribusjon** - Ingen behov for håndtering av legitimasjon
- ✅ **Revisjonsvennlig** - All tilgang spores gjennom Azure-identitet

Agenten vil bruke MCP-verktøy for å få tilgang til Microsoft Learn-ressurser med full sikkerhet og uten behov for API-nøkkelhåndtering.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
